In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots

In [2]:
using SparseArrays

In [3]:
#load the data and orgnize 
cap_cstr = CSV.read("/Users/bourg/.julia/environments/batterySC/data/capacity_constraint.csv", 
                    DataFrame, 
                    header=1) |> DataFrame

distance = CSV.File("/Users/bourg/.julia/environments/batterySC/data/distance.csv",header=1) |> DataFrame

LCA_model = CSV.File("/Users/bourg/.julia/environments/batterySC/data/LCA_model314.csv",header=1) |> DataFrame

SD = CSV.File("/Users/bourg/.julia/environments/batterySC/data/emission_sink.csv",header=1) |> DataFrame; 

In [4]:
cell_demand = 6000000 * 164.98/1000         # annual demand of Li battery for tesla (2M EV/yr, 3 NMC111 pack/EV, 164.98 kg/pack)
gobal_sink = 1.53e9                         # global CO2 sequestration (ton/yr)
global_emission = 4.75e10                   # global CO2 emission (ton/yr)
EF_aircraft = 0.433/1000                    # ton CO2/km*ton emission factor for freight transporation 
EF_input = LCA_model[!,"EF (kg CO2/material)"]
process = LCA_model[!,"process"]
countries = cap_cstr[!,"Country"]
n = size(countries,1)                       # No. of countries
m = size(process,1);                        # No. of processes 

In [5]:
# seperate model
# scaler_powder = LCA_model[1:4,"input (material/kg NMC111 powder)"]     # mineral inputs for 1 kg NMC111 powder
scaler_cell = LCA_model[1:10,"input (material/kg battery)"];     # inputs for 1 kg NMC111 powder
scaler_powder = scaler_cell[1:4] / scaler_cell[5]

4-element Vector{Float64}:
 0.03591269841269842
 0.20238095238095236
 0.19007936507936507
 0.20238095238095236

In [6]:
FU_input = cell_demand * LCA_model[1:11,"input (material/kg battery)"]

11-element Vector{Float64}:
   8958.413999999999
  50483.87999999999
  47415.25199999999
  50483.87999999999
 249449.75999999998
 139573.07999999996
  10888.679999999998
   3603.1631999999995
 115815.95999999999
 236581.31999999995
 989879.9999999999

----

In [7]:
model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only - expires 2023-11-27


In [8]:
#variables 
@variable(model, x[1:n, 1:m] >= 0)         # x[i,k] production amount of product k at location i
@variable(model, y[1:n, 1:n, 1:m] >= 0);   # y[i,j,k] ship product k from i to j 

In [9]:
# node output flow constraint
for k in 1:m
    for i in 1:n
        @constraint(model, sum(y[i,j,k] for j in 1:n) <= x[i,k])
    end
end

In [10]:
# NMC111 powder input flow constraint
for k in 1:4
    for j in 1:n
        @constraint(model, sum(y[i,j,k] for i in 1:n) >= x[j,5] * scaler_powder[k])
    end
end

# for j in 1:n
#     @constraint(model, sum(y[i,j,2] for i in 1:n) >= x[j,5] * scaler_powder[2])
# end

# for j in 1:n
#     @constraint(model, sum(y[i,j,1] for i in 1:n) >= x[j,5] * scaler_powder[1])
# end





# NMC111 battery input flow constraint
for k in 5:10
    for j in 1:n
        @constraint(model, sum(y[i,j,k] for i in 1:n) >= x[j,11] * scaler_cell[k])
    end
end



In [11]:
# production capacity constraints
for i in 1:n
    for k in 1:m
        @constraint(model, x[i,k] <= cap_cstr[!, 2:end][i,k])
    end
end

In [12]:
# final demand constraints
for k in 1:11
    @constraint(model, sum(x[i,k] for i in 1:n) == FU_input[k])
end

# @constraint(model, sum(x[i,11] for i in 1:n) == FU_battery);

In [13]:
vec_flow = []

for k in 1:m
    flow = sum(x[i, k] for i in 1:n)
    push!(vec_flow, flow)
end

D_proc = EF_input' * vec_flow;

In [14]:
trans_emi = 0

for k in 1:m
    for i in 1:n
        for j in 1:n
            trans_emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_aircraft
        end
    end
end

D_trans = trans_emi + sum(x[i,11] * distance[!, 2:end][i,38] for i in 1:n);

In [15]:
# supply - based on emission
emi = x * EF_input
S_country = SD[!, "sink ton/yr"] .* emi ./ SD[!, "emission"]
S_global = gobal_sink/global_emission * emi
S = S_global
S_tot = sum(S);

In [16]:
@objective(model, Min, D_trans+D_proc-S_tot);

In [17]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1291 rows, 18040 columns and 35320 nonzeros
Model fingerprint: 0x635b7834
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [2e-02, 2e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 1115 rows and 17283 columns
Presolve time: 0.02s

Solved with dual simplex
Solved in 48 iterations and 0.02 seconds
Optimal objective  7.472362676e+06

User-callback calls 41, time in user-callback 0.00 sec


-----

In [30]:
optx = copy(JuMP.value.(x))
opty = copy(JuMP.value.(y))
Li = optx[:, 1]
Co = optx[:, 2]
Mn = optx[:, 3]
Ni = optx[:, 4]
NMC = optx[:, 5];

In [25]:
for i in 1:n
    if Li_y[i]>0
        println(Li_y[i])
    end
end

In [183]:
copy(JuMP.value.(x))

40×11 Matrix{Float64}:
    0.0       0.0       0.0      0.0  …       0.0  0.0             0.0
    0.0    7400.0       0.0      0.0          0.0  0.0             0.0
    0.0       0.0       0.0      0.0          0.0  0.0             0.0
    0.0       0.0       0.0      0.0          0.0  0.0             0.0
    0.0       0.0       0.0      0.0          0.0  0.0             0.0
    0.0    4440.0       0.0      0.0  …       0.0  0.0             0.0
    0.0       0.0       0.0      0.0          0.0  0.0             0.0
 8958.41   2000.0       0.0      0.0          0.0  0.0             0.0
    0.0       0.0       0.0      0.0          0.0  0.0             0.0
    0.0   10400.0       0.0      0.0          0.0  0.0             0.0
    0.0    3500.0       0.0      0.0  …       0.0  0.0             0.0
    0.0    1380.0       0.0      0.0          0.0  0.0             0.0
    0.0       0.0       0.0      0.0          0.0  0.0             0.0
    ⋮                                 ⋱               

In [26]:
indices = CartesianIndices(opty)
nonzero_indices = indices[opty .!= 0]

println("Non-zero indices:")
for index in nonzero_indices
    println(index)
end


Non-zero indices:
CartesianIndex(8, 8, 1)
CartesianIndex(5, 38, 1)
CartesianIndex(6, 38, 1)
CartesianIndex(24, 38, 1)
CartesianIndex(30, 38, 1)
CartesianIndex(38, 38, 1)
CartesianIndex(2, 38, 2)
CartesianIndex(6, 38, 2)
CartesianIndex(8, 38, 2)
CartesianIndex(10, 38, 2)
CartesianIndex(11, 38, 2)
CartesianIndex(12, 38, 2)
CartesianIndex(18, 38, 2)
CartesianIndex(24, 38, 2)
CartesianIndex(25, 38, 2)
CartesianIndex(27, 38, 2)
CartesianIndex(29, 38, 2)
CartesianIndex(31, 38, 2)
CartesianIndex(33, 38, 2)
CartesianIndex(38, 38, 2)
CartesianIndex(39, 38, 2)
CartesianIndex(40, 38, 2)
CartesianIndex(24, 38, 3)
CartesianIndex(6, 38, 4)
CartesianIndex(38, 38, 5)
CartesianIndex(5, 38, 6)
CartesianIndex(6, 38, 6)
CartesianIndex(8, 38, 6)
CartesianIndex(15, 38, 6)
CartesianIndex(24, 38, 6)
CartesianIndex(26, 38, 6)
CartesianIndex(31, 38, 6)
CartesianIndex(35, 38, 6)
CartesianIndex(36, 38, 6)
CartesianIndex(38, 38, 7)
CartesianIndex(38, 38, 8)
CartesianIndex(38, 38, 9)
CartesianIndex(38, 38, 10)


In [51]:
using Tables
res = Tables.table(opty[1:end, 1:end, 11]);

In [52]:
CSV.write("/Users/bourg/Desktop/y11_matrix.csv", res) 

"/Users/bourg/Desktop/y11_matrix.csv"